In [48]:
import pandas as pd
import os
import sys
import re

In [49]:
file_name2= r"/Users/suranjanachowdhury/Downloads/2022_11_23_ESG_funds.csv"
data_raw=pd.read_csv(file_name2)
print(data_raw.shape)

(3595, 18)


In [50]:
data_raw.head()

,Unnamed: 0,accession,filing_date,fund_family_name,fund_name,principal_strategies,principal_risks,fund_cik,stock_series,business_address,mailing_address,filing_year,series_cik_1,crsp_obj_cd,crsp_class,fund_family_name_lower_case,fund_name_lower_case,esg_fund
0,60,0000930413-10-002588,20100430,TIAA CREF LIFE FUND,Social Choice Equity Fund,NaN,NaN,1068204,s000005095,"730 THIRD AVE, NEW YORK, NY, 10017, 2124909000","730 THIRD AVE, NEW YORK, NY, 10017",2010,s000005095,EDYB,DE,tiaa cref life fund,social choice equity fund,fund_name_keyword
1,175,0000930413-10-004557,20100823,TIAA CREF LIFE FUND,Social Choice Equity Fund,"Under normal circumstances, the Fund inves...",You could lose money over short or long period...,1068204,s000005095,"730 THIRD AVE, NEW YORK, NY, 10017, 2124909000","730 THIRD AVE, NEW YORK, NY, 10017",2010,s000005095,EDYB,DE,tiaa cref life fund,social choice equity fund,fund_name_keyword
2,176,0000930413-10-004566,20100823,TIAA-CREF FUNDS,Social Choice Equity Fund,"Under normal circumstances, the Fund invests a...",You could lose money over short or long period...,1084380,s000005371,"730 THIRD AVE., NEW YORK, NY, 10017, 2129166746","730 THIRD AVE., NEW YORK, NY, 10017",2010,s000005371,EDYG,DE,tiaa-cref funds,social choice equity fund,fund_name_keyword
3,583,0000932471-10-003644,20101215,VANGUARD WORLD FUND,Vanguard FTSE Social Index Fund,NaN,NaN,52848,s000004440,"PO BOX 2600, V26, VALLEY FORGE, PA, 19482, 610...","PO BOX 2600, V26, VALLEY FORGE, PA, 19482",2010,s000004440,EDYG,DE,vanguard world fund,vanguard ftse social index fund,fund_name_keyword
4,596,0000932471-10-003657,20101215,VANGUARD WORLD FUND,Vanguard FTSE Social Index Fund,NaN,NaN,52848,s000004440,"PO BOX 2600, V26, VALLEY FORGE, PA, 19482, 610...","PO BOX 2600, V26, VALLEY FORGE, PA, 19482",2010,s000004440,EDYG,DE,vanguard world fund,vanguard ftse social index fund,fund_name_keyword


In [51]:
#lets check unique fund names
num_unique_values = data_raw['fund_name'].nunique()
print("Number of unique values in 'column_name':", num_unique_values)

Number of unique values in 'column_name': 492


In [52]:
import nltk

In [53]:
sent_col_name = 'principal_strategies' #first strategy
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: str(x).replace("&nbsp;", " "))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\\", ''))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\t", ''))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\n", ''))

In [54]:
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [55]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [56]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    
    #getting all the keys for abbrevations
    contractions = abbreviations.keys()
    
    #making the list of sentence terminators
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    
    for sentence_terminator in sentence_terminators:
    
        #Finding the position of sentence terminators
        t_indices = list(find_all(paragraph, sentence_terminator))
        
        #Finding the length of sentence terminators
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    
    for contraction in contractions:
        
        #Finding the position of abbrevations defined above
        c_indices = list(find_all(paragraph, contraction))
        
        #Finding the end of abbrevations in a disclosure                  
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    
    #tokenizing the sentences from paragraph
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    
    return end

In [57]:
def find_sentences(paragraph):
    
    replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], 
                     ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], 
                     ["S. E. C.", "S.E.C."]]
    
    for items in replacingList:
        #adding space or cleaning the common abbrevations
        paragraph = paragraph.replace(items[0], items[1])
    
    #replacing all the extra spaces with single space    
    paragraph = re.sub(' +', ' ', paragraph)
    
    #removing the abbrevations which are used to denote staement ending from the beginning of the statement
    if paragraph != "":
        if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
            paragraph = paragraph[1:]    
    
    end = True
    sentences = []
    
    while end > -1:
        end = find_sentence_end(paragraph)
        if end > -1:
            sentences.append(paragraph[end:].strip())
            paragraph = paragraph[:end]
    
    sentences.append(paragraph)
    sentences.reverse()
    
    return sentences

In [58]:
#import nltk.data
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data_raw['Integration_Strategy']= ""

for row in data_raw.itertuples():
    if row.Index %5000 == 0:
        print ("processing row: ", row.Index)    
        
    para = row.principal_strategies
    para = para.replace('{',' ')
    para = para.replace('}',' ')
    sents = find_sentences(para)
    text = ' '.join(sents)
    text = re.sub('[^,.a-zA-Z0-9 ]',' ', text)
    text = '\n'.join(tokenizer.tokenize(text))
    data_raw['Integration_Strategy'].loc[row.Index] = text

processing row:  0


/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_4108/1591655150.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_raw['Integration_Strategy'].loc[row.Index] = text


In [59]:
innovation_words = ['impact', 'clean', 'energy', 'alternative','alternatives','water','sustainable',
                    'sustainability','social','socially','responsible','environmental','environment',
                    'environmentally','ESG','social','governance','index' ]

In [60]:
import nltk
from nltk.corpus import stopwords
import re
import numpy as np
import os
import pandas as pd
import glob
import nltk.data
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters, PunktTrainer

In [61]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/suranjanachowdhury/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
# Tokenizing disclosure text
data_raw['tokens'] = data_raw.apply(lambda row: nltk.sent_tokenize(row['Integration_Strategy']), axis=1)

In [63]:
data_raw.head()

,Unnamed: 0,accession,filing_date,fund_family_name,fund_name,principal_strategies,principal_risks,fund_cik,stock_series,business_address,mailing_address,filing_year,series_cik_1,crsp_obj_cd,crsp_class,fund_family_name_lower_case,fund_name_lower_case,esg_fund,Integration_Strategy,tokens
0,60,0000930413-10-002588,20100430,TIAA CREF LIFE FUND,Social Choice Equity Fund,nan,NaN,1068204,s000005095,"730 THIRD AVE, NEW YORK, NY, 10017, 2124909000","730 THIRD AVE, NEW YORK, NY, 10017",2010,s000005095,EDYB,DE,tiaa cref life fund,social choice equity fund,fund_name_keyword,nan,[nan]
1,175,0000930413-10-004557,20100823,TIAA CREF LIFE FUND,Social Choice Equity Fund,"Under normal circumstances, the Fund inves...",You could lose money over short or long period...,1068204,s000005095,"730 THIRD AVE, NEW YORK, NY, 10017, 2124909000","730 THIRD AVE, NEW YORK, NY, 10017",2010,s000005095,EDYB,DE,tiaa cref life fund,social choice equity fund,fund_name_keyword,Under normal circumstances the Fund invests at...,[Under normal circumstances the Fund invests a...
2,176,0000930413-10-004566,20100823,TIAA-CREF FUNDS,Social Choice Equity Fund,"Under normal circumstances, the Fund invests a...",You could lose money over short or long period...,1084380,s000005371,"730 THIRD AVE., NEW YORK, NY, 10017, 2129166746","730 THIRD AVE., NEW YORK, NY, 10017",2010,s000005371,EDYG,DE,tiaa-cref funds,social choice equity fund,fund_name_keyword,Under normal circumstances the Fund invests at...,[Under normal circumstances the Fund invests a...
3,583,0000932471-10-003644,20101215,VANGUARD WORLD FUND,Vanguard FTSE Social Index Fund,nan,NaN,52848,s000004440,"PO BOX 2600, V26, VALLEY FORGE, PA, 19482, 610...","PO BOX 2600, V26, VALLEY FORGE, PA, 19482",2010,s000004440,EDYG,DE,vanguard world fund,vanguard ftse social index fund,fund_name_keyword,nan,[nan]
4,596,0000932471-10-003657,20101215,VANGUARD WORLD FUND,Vanguard FTSE Social Index Fund,nan,NaN,52848,s000004440,"PO BOX 2600, V26, VALLEY FORGE, PA, 19482, 610...","PO BOX 2600, V26, VALLEY FORGE, PA, 19482",2010,s000004440,EDYG,DE,vanguard world fund,vanguard ftse social index fund,fund_name_keyword,nan,[nan]


In [64]:
keyword_sentence = []


for row in data_raw.itertuples():
    if row.Index %5000 == 0:
        print ("processing row: ", row.Index)        
   
    for j in row.tokens:
        for k in innovation_words:
            if k in j:
#               print(row.accession,row.filing_date, row.crsp_class, row.filing_year, j)
                #keyword_sentence.append([row.accession,row.filing_date, row.crsp_class, row.filing_year, j])
                keyword_sentence.append([row.accession, 
                                         row.filing_date,
                                         row.fund_name,
                                         row.fund_cik,
                                         #row.crsp_fundno, 
                                         #row.crsp_portno, 
                                         #row.fund_name, 
                                         row.principal_strategies,
                                         row.stock_series,
                                         #row.comp_cik,                                      
                                         #row.cik,
                                         j,
                                         k]) 
 

processing row:  0


In [65]:
print(len(keyword_sentence))

43122


In [66]:
df_sent = pd.DataFrame(keyword_sentence, columns =['accession', 
                                         'filing_date',
                                         'fund_name',
                                          'fund_cik',
                                         #row.crsp_fundno, 
                                         #row.crsp_portno, 
                                         #row.fund_name, 
                                         'principal_strategies' ,         
                                         'stock_series',
                                         'sentences',
                                         'Name_keyword'])#replacing here


In [67]:
df_sent['Name_keyword'].value_counts() #check distribution

social             9686
ESG                8656
environment        4195
index              3926
environmental      3619
governance         3467
sustainable        2278
responsible        1770
sustainability     1478
impact             1336
socially            843
energy              839
water               412
alternative         283
clean               224
alternatives         56
environmentally      54
Name: Name_keyword, dtype: int64

In [68]:
df_sent.shape

(43122, 8)

In [69]:
df_sent.head()

,accession,filing_date,fund_name,fund_cik,principal_strategies,stock_series,sentences,Name_keyword
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social
1,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social
3,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social


In [70]:
df_sent_unique = df_sent.drop_duplicates(subset =['sentences'])
print(df_sent_unique.shape)
df_sent_unique.head()

(3828, 8)


,accession,filing_date,fund_name,fund_cik,principal_strategies,stock_series,sentences,Name_keyword
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social
10,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund does this by investing in U.S. compan...,ESG
11,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,Prior to being eligible for inclusion in the M...,ESG


In [71]:
data=df_sent_unique

# Selecting fund names and checking

In [101]:
sent_col_name = 'fund_name' #first strategy
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: str(x).replace("&nbsp;", " "))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\\", ''))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\t", ''))
data_raw[sent_col_name]=data_raw[sent_col_name].apply(lambda x: x.replace("\n", ''))

In [102]:
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [103]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [104]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    
    #getting all the keys for abbrevations
    contractions = abbreviations.keys()
    
    #making the list of sentence terminators
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    
    for sentence_terminator in sentence_terminators:
    
        #Finding the position of sentence terminators
        t_indices = list(find_all(paragraph, sentence_terminator))
        
        #Finding the length of sentence terminators
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    
    for contraction in contractions:
        
        #Finding the position of abbrevations defined above
        c_indices = list(find_all(paragraph, contraction))
        
        #Finding the end of abbrevations in a disclosure                  
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    
    #tokenizing the sentences from paragraph
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    
    return end

In [105]:
def find_sentences(paragraph):
    
    replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], 
                     ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], 
                     ["S. E. C.", "S.E.C."]]
    
    for items in replacingList:
        #adding space or cleaning the common abbrevations
        paragraph = paragraph.replace(items[0], items[1])
    
    #replacing all the extra spaces with single space    
    paragraph = re.sub(' +', ' ', paragraph)
    
    #removing the abbrevations which are used to denote staement ending from the beginning of the statement
    if paragraph != "":
        if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
            paragraph = paragraph[1:]    
    
    end = True
    sentences = []
    
    while end > -1:
        end = find_sentence_end(paragraph)
        if end > -1:
            sentences.append(paragraph[end:].strip())
            paragraph = paragraph[:end]
    
    sentences.append(paragraph)
    sentences.reverse()
    
    return sentences

In [106]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

data_raw['Fund_Strategy']= ""

for row in data_raw.itertuples():
    if row.Index %5000 == 0:
        print ("processing row: ", row.Index)    
        
    para = row.principal_strategies
    para = para.replace('{',' ')
    para = para.replace('}',' ')
    sents = find_sentences(para)
    text = ' '.join(sents)
    text = re.sub('[^,.a-zA-Z0-9 ]',' ', text)
    text = '\n'.join(tokenizer.tokenize(text))
    data_raw['Fund_Strategy'].loc[row.Index] = text

processing row:  0


/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_4108/2760873926.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_raw['Fund_Strategy'].loc[row.Index] = text


In [107]:
# Tokenizing disclosure text
data_raw['tokens'] = data_raw.apply(lambda row: nltk.sent_tokenize(row['Fund_Strategy']), axis=1)

In [108]:
keyword_sentence = []


for row in data_raw.itertuples():
    if row.Index %5000 == 0:
        print ("processing row: ", row.Index)        
   
    for j in row.tokens:
        for k in innovation_words:
            if k in j:
#               print(row.accession,row.filing_date, row.crsp_class, row.filing_year, j)
                #keyword_sentence.append([row.accession,row.filing_date, row.crsp_class, row.filing_year, j])
                keyword_sentence.append([row.accession, 
                                         row.filing_date,
                                         row.fund_name,
                                         row.fund_cik,
                                         #row.crsp_fundno, 
                                         #row.crsp_portno, 
                                         #row.fund_name, 
                                         row.principal_strategies,
                                         row.stock_series,
                                         #row.comp_cik,                                      
                                         #row.cik,
                                         j,
                                         k]) 
 

processing row:  0


In [109]:
df_sent = pd.DataFrame(keyword_sentence, columns =['accession', 
                                         'filing_date',
                                         'fund_name',
                                          'fund_cik',
                                         #row.crsp_fundno, 
                                         #row.crsp_portno, 
                                         #row.fund_name, 
                                         'fund_principal_strategies' ,         
                                         'stock_series',
                                         'sentences',
                                         'Name_keyword'])#replacing here


In [110]:
df_sent['Name_keyword'].value_counts() #check distribution

social             9686
ESG                8656
environment        4195
index              3926
environmental      3619
governance         3467
sustainable        2278
responsible        1770
sustainability     1478
impact             1336
socially            843
energy              839
water               412
alternative         283
clean               224
alternatives         56
environmentally      54
Name: Name_keyword, dtype: int64

In [111]:
df_sent.shape

(43122, 8)

In [112]:
df_sent_unique = df_sent.drop_duplicates(subset =['sentences'])
print(df_sent_unique.shape)
df_sent_unique.head()

(3828, 8)


,accession,filing_date,fund_name,fund_cik,fund_principal_strategies,stock_series,sentences,Name_keyword
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social
10,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund does this by investing in U.S. compan...,ESG
11,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,Prior to being eligible for inclusion in the M...,ESG


In [113]:
data=df_sent_unique

In [114]:
data.head()

,accession,filing_date,fund_name,fund_cik,fund_principal_strategies,stock_series,sentences,Name_keyword
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social
10,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund does this by investing in U.S. compan...,ESG
11,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,Prior to being eligible for inclusion in the M...,ESG


# Labelling Code(Created sublabels)

In [128]:
#data.drop('Unnamed: 0', axis=1, inplace=True)
data.head()

,accession,filing_date,fund_name,fund_cik,fund_principal_strategies,stock_series,sentences,Name_keyword,name_hint
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social,integration
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social,integration
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social,integration
10,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund does this by investing in U.S. compan...,ESG,focus
11,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,Prior to being eligible for inclusion in the M...,ESG,focus


In [129]:
all_series = data['stock_series'].unique()


In [130]:
print(len(all_series))

350


In [131]:
def categorize_value(label1):
                if label1 == 'impact':
                    return 'impact'
                elif label1 == 'sustainable' or label1 == 'sustainability' or label1 == 'social' or label1 == 'socially' or label1 == 'responsible':
                    return 'integration'
                elif label1 == 'environmental' or label1 == 'environment' or label1 == 'environmentally' or label1 == 'ESG' or label1 == 'social' or label1 == 'governance' or label1 == 'index' or label1 == 'clean' or label1 == 'energy' or label1 == 'alternative' or label1 == 'alternatives' or label1 == 'water':
                    return 'focus' 
                else:
                    return 'residual'


In [132]:
data['name_hint'] = data['Name_keyword'].apply(categorize_value) 

/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_4108/1197967112.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['name_hint'] = data['Name_keyword'].apply(categorize_value)


In [133]:
data.head()

,accession,filing_date,fund_name,fund_cik,fund_principal_strategies,stock_series,sentences,Name_keyword,name_hint
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social,integration
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social,integration
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social,integration
10,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund does this by investing in U.S. compan...,ESG,focus
11,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,Prior to being eligible for inclusion in the M...,ESG,focus


In [134]:
data.shape

(3828, 9)

In [135]:
data['name_hint'].value_counts()

focus          2264
integration    1289
impact          275
Name: name_hint, dtype: int64

In [136]:
data['Name_keyword'].value_counts()

ESG               1320
social             638
index              429
sustainable        340
impact             275
sustainability     211
energy             172
responsible        100
clean               88
water               79
environmental       73
environment         38
governance          36
alternative         29
Name: Name_keyword, dtype: int64

In [137]:
#Part 3 ,secendary labelling and putting tags,5 word window

In [138]:
#part 3,4,5

In [139]:
import pandas as pd
import numpy as np

def extract_window(s, keyword):
    """
    Given a string s and a keyword, extract a 5-word window before and after the keyword
    and return the result as a string.
    """
    words = s.split()
    try:
        index = words.index(keyword)
    except ValueError:
        return ""
    start = max(index-5, 0)
    end = min(index+6, len(words))
    return " ".join(words[start:end])

def extract_integration_strategy(df, keyword_list):
    """
    Given a pandas DataFrame and a list of keywords, extract a 5-word window before and after each keyword
    for each value in the "principal_strategy" column that matches a keyword in the "name_hint" column.
    Create a new column for each keyword called "<Keyword>_Strategies".
    Replace empty strings with NaN values.
    """
    for keyword in keyword_list:
        mask = df['name_hint'].str.contains(keyword)
        column_name = f"{keyword.capitalize()}_Strategies"
        df.loc[mask, column_name] = df['fund_name'].apply(lambda s: extract_window(s, keyword))
        df[column_name].replace("", np.nan, inplace=True)
    return df


In [140]:
keyword_list = ['integration', 'impact','focus']
data = extract_integration_strategy(data, keyword_list)


/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_4108/2731763095.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[mask, column_name] = df['fund_name'].apply(lambda s: extract_window(s, keyword))
/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_4108/2731763095.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name].replace("", np.nan, inplace=True)
/var/folders/xx/rbbr4jzx5d139mngn53bcc4c0000gn/T/ipykernel_4108/2731763095.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
T

In [141]:
data.head()

,accession,filing_date,fund_name,fund_cik,fund_principal_strategies,stock_series,sentences,Name_keyword,name_hint,Integration_Strategies,Impact_Strategies,Focus_Strategies
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social,integration,NaN,NaN,NaN
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social,integration,NaN,NaN,NaN
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social,integration,NaN,NaN,NaN
10,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund does this by investing in U.S. compan...,ESG,focus,NaN,NaN,NaN
11,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,Prior to being eligible for inclusion in the M...,ESG,focus,NaN,NaN,NaN


In [145]:
data.isna().sum()

accession                       0
filing_date                     0
fund_name                       0
fund_cik                        0
fund_principal_strategies       0
stock_series                    0
sentences                       0
Name_keyword                    0
name_hint                       0
Integration_Strategies       3828
Impact_Strategies            3828
Focus_Strategies             3828
dtype: int64

In [154]:
import pandas as pd
import regex

if 'integration' in data['name_hint']:
    pat = r'also takes'
    data['Integration_Strategies'] = data['sentences'].apply(lambda x: 1 if regex.search(pat, x) else 0)


In [155]:
data.head()

,accession,filing_date,fund_name,fund_cik,fund_principal_strategies,stock_series,sentences,Name_keyword,name_hint,Integration_Strategies,Impact_Strategies,Focus_Strategies
0,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund attempts to track the return of the U...,social,integration,NaN,NaN,NaN
2,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The social criteria the Fund takes into consid...,social,integration,NaN,NaN,NaN
4,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund primarily invests in companies that a...,social,integration,NaN,NaN,NaN
10,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,The Fund does this by investing in U.S. compan...,ESG,focus,NaN,NaN,NaN
11,0000930413-10-004557,20100823,Social Choice Equity Fund,1068204,"Under normal circumstances, the Fund inves...",s000005095,Prior to being eligible for inclusion in the M...,ESG,focus,NaN,NaN,NaN


In [156]:
if data['Integration_Strategies'].notna().any():
    print("Integration strategies contains values other than NaN.")
else:
    print("Integration strategies does not contain any values other than NaN.")

Integration strategies does not contain any values other than NaN.


In [144]:
data.to_excel('dataset_IFIR_FundName.xlsx')